In [1]:
import pandas as pd

In [2]:
#Load business file

df_b = pd.read_json("yelp_academic_dataset_business.json", lines=True)

In [3]:
#First five rows
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


In [4]:
#Clean Business File
df_b = df_b[df_b['is_open']==1].drop(["postal_code", "latitude", "longitude", "hours"], axis=1)

In [5]:
#List categories
df_explode = df_b.assign(categories = df_b.categories
                         .str.split(', ')).explode('categories')
df_explode.categories.value_counts()[:15]

Restaurants                  32022
Food                         20418
Shopping                     20309
Home Services                14795
Health & Medical             13630
Beauty & Spas                13569
Local Services               10677
Automotive                    8886
Event Planning & Services     7856
Nightlife                     7829
Active Life                   7675
Bars                          7008
Coffee & Tea                  5476
Hotels & Travel               5032
Sandwiches                    4795
Name: categories, dtype: int64

In [6]:
#List all categories with restaurant
df_explode[df_explode.categories.str.contains('Restaurants',
                      case=True,na=False)].categories.value_counts()

Restaurants           32022
Pop-Up Restaurants       48
Name: categories, dtype: int64

In [7]:
#Keep only restaurants in British Columbia
restaurant = df_b[df_b['categories'].str.contains(
              'Restaurants',
              case=False, na=False)]
print("Before State {}".format(restaurant.shape))
restaurant =  restaurant[restaurant['state'].str.contains(
              'BC',
              case=False, na=False)]
print("After State {}".format(restaurant.shape))

Before State (32022, 10)
After State (4728, 10)


In [8]:
#Keep only restaurants in Vancouver
print("Before City {}".format(restaurant.shape))
restaurant = restaurant[restaurant['city'].str.contains(
              'Vancouver',
              case=False, na=False)]
print("After City {}".format(restaurant.shape))

Before City (4728, 10)
After City (2798, 10)


In [9]:
#Keep only sushi restaurants
print("Before Sushi {}".format(restaurant.shape))
restaurant = restaurant[restaurant['categories'].str.contains(
             'Sushi',
              case=False, na=False)]
print("After Sushi {}".format(restaurant.shape))

Before Sushi (2798, 10)
After Sushi (235, 10)


In [10]:
#Keep only sushi restaurants
print("Before review count {}".format(restaurant.shape))
restaurant = restaurant[restaurant['review_count'] >= 100]
print("After review count {}".format(restaurant.shape))

Before review count (235, 10)
After review count (72, 10)


In [11]:
#Loading review file
size = 1000000
df_review = pd.read_json("yelp_academic_dataset_review.json", lines=True,dtype={'review_id':str,'user_id':str,'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,'funny':int,'cool':int},
                      chunksize=size)

In [12]:
# There are multiple chunks to be read
chunk_list = []
for chunk in df_review:
    # Drop columns that aren't needed
    chunk = chunk.drop(['useful','funny','cool', 'date'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk = chunk.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(restaurant, chunk, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)

1945 out of 1,000,000 related reviews
857 out of 1,000,000 related reviews
1375 out of 1,000,000 related reviews
3989 out of 1,000,000 related reviews
2805 out of 1,000,000 related reviews
1436 out of 1,000,000 related reviews
2667 out of 1,000,000 related reviews
1212 out of 1,000,000 related reviews
1637 out of 1,000,000 related reviews


In [13]:
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
df.sample(3)

,business_id,name,address,city,state,stars,review_count,is_open,attributes,categories,review_id,user_id,review_stars,text
8502,4EV_ZcQmjAmP3pmO-_nb2A,Miku,"200 Granville Street, Suite 70",Vancouver,BC,4.5,1805,1,"{'RestaurantsAttire': ''casual'', 'BikeParking...","Japanese, Sushi Bars, Restaurants",q8hSvsGH8RZIIFvWPNxwgA,AkQlUUSHD0UFz1XuEqZryA,5,What a treat! Went here last year to splurge o...
16809,8HH7xRjmjwzkhADZWir_Cw,Black Rice Izakaya,782 Cambie Street,Vancouver,BC,4.0,238,1,"{'WiFi': 'u'free'', 'RestaurantsPriceRange2': ...","Tapas Bars, Restaurants, Tapas/Small Plates, J...",K3CNaULOP2hZADOk9j-fxQ,mNYQl3d2wFDWZiZa7VD6og,4,I wanted Aburi sushi for dinner and saw on Yel...
12842,I0r8kMimYW2BY6lINcZRFA,Kishimoto Japanese Kitchen,2054 Commercial Drive,Vancouver,BC,4.5,469,1,"{'HasTV': 'False', 'WiFi': 'u'no'', 'Alcohol':...","Restaurants, Sushi Bars, Japanese",W-ZJM7i0fLpQrvfFEcQF4Q,ZsErYjyHiW0QBMrBH7qNUA,5,I accidentally found this gem. Who would think...


In [14]:
#Convert dataframe to csv
df.to_csv("yelp_reviews_restaurants.csv", index=False)
